In [1]:
import pymc as pm
import numpy as np
import aesara
import aesara.tensor as at
import arviz as az
# Simulate some data
N_samples = 100
p_NaN = 0.1
rng = np.random.RandomState(123)
x = rng.normal(10., 2., size=N_samples)
x[rng.rand(N_samples) < p_NaN] = np.nan

a, b = 2., 1.
sigma = 2.
y = a*x + b + rng.normal(0.0, sigma, size=N_samples)
y[np.isnan(y)] = 0.0

WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
x_b = np.ones_like(x)
x_copy = x.copy()
x_copy[np.isnan(x)] = 0.
x_b[np.isnan(x)] = 0.

In [3]:
sigma_fixed = aesara.shared(.1)

with pm.Model() as model_1:
    a = pm.Normal('a', 0.0, 1.0)
    b = pm.Normal('b', 0.0, 1.0)
    sigma = pm.Exponential('sigma', 1.0)
    sigma_ = at.stack([sigma, sigma_fixed])
    mu = a * x_copy + b * x_b
    y_normal = pm.Normal('y_normal', mu, sigma_[np.isnan(x).astype(int)], observed=y)
    idata = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a, b, sigma]
WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 22 seconds.
The acceptance probability does not match the target. It is 0.8994, but should be close to 0.8. Try to increase the number of tuning steps.
/opt/miniconda3/envs/aesara-dev/lib/python3.9/site-packages/numba/core/dispatcher.py:241: UserWarning: Numba extension module 'numba_scipy' failed to load due to 'ValueError(No function '__pyx_fuse_0pdtr' found in __pyx_capi__ of 'scipy.special.cython_special')'.
  entrypoints.init_all()


In [4]:
az.summary(idata)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
a,2.021,0.073,1.891,2.163,0.002,0.001,1340.0,1584.0,1.01
b,0.352,0.719,-0.885,1.769,0.020,0.014,1362.0,1525.0,1.00
sigma,2.006,0.151,1.736,2.287,0.004,0.003,1810.0,1825.0,1.00
